# Parsing HTML Files for Data

### Libraries

In [2]:
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup, Comment
import re
import os
from pathlib import Path

## Box Scores

### Game File Location

In [ ]:
games_path = Path("/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/AllGames")
game_data = []

### Scraping Basic Box Score Data

In [ ]:
basic_dfs = []

for game in os.listdir(games_path):
    with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/AllGames/{game}", encoding="utf-8") as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")

    basic_tables = soup.find_all(id=re.compile(r"^box-.*-game-basic$"))

    table_dfs = []
    for table in basic_tables:
        table_id = table.get("id")
        table_html = str(table)
        df = pd.read_html(StringIO(table_html), header=1)[0]
        df["team_table_id"] = table_id
        df["game_id"] = game
        table_dfs.append(df)

    # Add the opposing team table id
    if len(table_dfs) == 2:
        table_dfs[0]["opp_team_table_id"] = table_dfs[1]["team_table_id"]
        table_dfs[1]["opp_team_table_id"] = table_dfs[0]["team_table_id"]

    basic_dfs.extend(table_dfs)

basic_tables_combined = pd.concat(basic_dfs, ignore_index=True)

In [ ]:
basic_tables_combined

#### Exporting Basic Box Score to CSV File

In [ ]:
basic_tables_combined.to_csv("/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/BasicBox.csv")

### Scraping Advanced Box Score Data

In [ ]:
advanced_dfs = []

for game in os.listdir(games_path):
    with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/AllGames/{game}", encoding="utf-8") as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")

    advanced_tables = soup.find_all(id=re.compile(r"^box-.*-game-advanced$"))

    table_dfs = []
    for table in advanced_tables:
        table_id = table.get("id")
        table_html = str(table)
        df = pd.read_html(StringIO(table_html), header=1)[0]
        df["team_table_id"] = table_id
        df["game_id"] = game
        table_dfs.append(df)

    # Add the opposing team table id
    if len(table_dfs) == 2:
        table_dfs[0]["opp_team_table_id"] = table_dfs[1]["team_table_id"]
        table_dfs[1]["opp_team_table_id"] = table_dfs[0]["team_table_id"]

    advanced_dfs.extend(table_dfs)

advanced_tables_combined = pd.concat(advanced_dfs, ignore_index=True)

In [ ]:
advanced_tables_combined

#### Exporting Advanced Box Scores to CSV File

In [ ]:
advanced_tables_combined.to_csv("/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/AdvancedBox.csv")

## Team Stats

### IDs to Collect
* Team Misc: team_misc
* Team Per Game: per_game_stats, per_game_stats
* Team Totals: totals_stats, totals_stats_post
* Per 36 Min: per_minute_stats, per_minute_stats_post
* Per 100 Poss: per_poss, per_poss_post
* Advanced: advanced, advanced_post
* Adjusted Shooting: adj_shooting, adj_shooting_post
* Shooting: shooting, shooting_post
* Play by Play: pbp_stats, pbp_stats_post

### Scrape Type
* Regular
    * Per Game
    * Per 36 Min
* In Comments
    * Team Totals
    * Per 100 Poss
    * Advanced
    * Adjusted Shooting
    * Shooting
    * Play by Play
    * Team Misc

In [3]:
all_teams = Path("/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/All_Teams")
playoff_teams = Path("/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/Playoff_Teams")

#### Team and Opponent Stats

In [4]:
all_general_dfs = []

for team in os.listdir(all_teams):
    if team.endswith(".html"):
        with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/All_Teams/{team}", encoding="ISO-8859-1") as f:
            page = f.read()
        
        soup = BeautifulSoup(page, "html.parser")

        comment = next(c for c in soup.find_all(string=lambda text: isinstance(text, Comment)) if "team_and_opponent" in c)
        table = BeautifulSoup(comment, "html.parser").find(id="team_and_opponent")
        df = pd.read_html(StringIO(str(table)))[0]
        df["team_id"] = team
        all_general_dfs.append(df)

all_general_dfs = pd.concat(all_general_dfs, ignore_index=True)

In [6]:
all_general_dfs.to_csv("/Users/camsmithers/Desktop/Camalytics/NBA/Data-NBA/all_general.csv")

#### Team Misc

In [ ]:
all_misc_dfs = []

for team in os.listdir(all_teams):
    if team.endswith(".html"):
        with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/All_Teams/{team}", encoding="ISO-8859-1") as f:
            page = f.read()
        
        soup = BeautifulSoup(page, "html.parser")

        comment = next(c for c in soup.find_all(string=lambda text: isinstance(text, Comment)) if "team_misc" in c)
        table = BeautifulSoup(comment, "html.parser").find(id="team_misc")
        df = pd.read_html(StringIO(str(table)), header=1)[0]
        df["team_id"] = team
        all_misc_dfs.append(df)

all_misc_combined = pd.concat(all_misc_dfs, ignore_index=True)

#### Per Game Stats

In [ ]:
all_pgs_dfs = []

for team in os.listdir(all_teams):
    with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/All_Teams/{team}", encoding="ISO-8859-1") as f:
        page = f.read()
    
    soup = BeautifulSoup(page, "html.parser")

    tables = soup.find_all(id="per_game_stats")

    for table in tables:
        table_html = str(table)
        df = pd.read_html(StringIO(table_html))[0]
        df["team_id"] = team
        all_pgs_dfs.append(df)
all_pgs_combined = pd.concat(all_pgs_dfs, ignore_index=True)

all_pgs_combined

In [ ]:
playoff_pgs_dfs = []

for team in os.listdir(playoff_teams):
    with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/Playoff_Teams/{team}", encoding="ISO-8859-1") as f:
        page = f.read()
    
    soup = BeautifulSoup(page, "html.parser")

    tables = soup.find_all(id="per_game_stats_post")

    for table in tables:
        table_html = str(table)
        df = pd.read_html(StringIO(table_html))[0]
        df["team_id"] = team
        playoff_pgs_dfs.append(df)
playoff_pgs_combined = pd.concat(playoff_pgs_dfs, ignore_index=True)

playoff_pgs_combined

#### Team Totals

In [ ]:
all_totals_dfs = []

for team in os.listdir(all_teams):
    if team.endswith(".html"):
        with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/All_Teams/{team}", encoding="ISO-8859-1") as f:
            page = f.read()
        
        soup = BeautifulSoup(page, "html.parser")

        comment = next(c for c in soup.find_all(string=lambda text: isinstance(text, Comment)) if "totals_stats" in c)
        table = BeautifulSoup(comment, "html.parser").find(id="totals_stats")
        df = pd.read_html(StringIO(str(table)))[0]
        df["team_id"] = team
        all_totals_dfs.append(df)

all_totals_combined = pd.concat(all_totals_dfs, ignore_index=True)

In [ ]:
playoff_totals_dfs = []

for team in os.listdir(playoff_teams):
    if team.endswith(".html"):
        with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/Playoff_Teams/{team}", encoding="ISO-8859-1") as f:
            page = f.read()
        
        soup = BeautifulSoup(page, "html.parser")

        comment = next(c for c in soup.find_all(string=lambda text: isinstance(text, Comment)) if "totals_stats_post" in c)
        table = BeautifulSoup(comment, "html.parser").find(id="totals_stats_post")
        df = pd.read_html(StringIO(str(table)))[0]
        df["team_id"] = team
        playoff_totals_dfs.append(df)

playoff_totals_combined = pd.concat(playoff_totals_dfs, ignore_index=True)

#### Per 36 Min

In [ ]:
all_per36_dfs = []

for team in os.listdir(all_teams):
    if team.endswith(".html"):
        with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/All_Teams/{team}", encoding="ISO-8859-1") as f:
            page = f.read()
        
        soup = BeautifulSoup(page, "html.parser")

        tables = soup.find_all(id="per_minute_stats")

        for table in tables:
            table_html = str(table)
            df = pd.read_html(StringIO(table_html))[0]
            df["team_id"] = team
            all_per36_dfs.append(df)

all_per36_combined = pd.concat(all_per36_dfs, ignore_index=True)

all_per36_combined

In [ ]:
playoff_per36_dfs = []

for team in os.listdir(playoff_teams):
    if team.endswith(".html"):
        with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/Playoff_Teams/{team}", encoding="ISO-8859-1") as f:
            page = f.read()
        
        soup = BeautifulSoup(page, "html.parser")

        tables = soup.find_all(id="per_minute_stats_post")

        for table in tables:
            table_html = str(table)
            df = pd.read_html(StringIO(table_html))[0]
            df["team_id"] = team
            playoff_per36_dfs.append(df)

playoff_per36_combined = pd.concat(playoff_per36_dfs, ignore_index=True)

playoff_per36_combined

#### Per 100 Poss

In [ ]:
all_per100_dfs = []

for team in os.listdir(all_teams):
    if team.endswith(".html"):
        with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/All_Teams/{team}", encoding="ISO-8859-1") as f:
            page = f.read()
        
        soup = BeautifulSoup(page, "html.parser")

        comment = next(c for c in soup.find_all(string=lambda text: isinstance(text, Comment)) if "per_poss" in c)
        table = BeautifulSoup(comment, "html.parser").find(id="per_poss")
        df = pd.read_html(StringIO(str(table)))[0]
        df["team_id"] = team
        all_per100_dfs.append(df)

all_per100_combined = pd.concat(all_per100_dfs, ignore_index=True)

all_per100_combined

In [ ]:
playoff_per100_dfs = []

for team in os.listdir(playoff_teams):
    if team.endswith(".html"):
        with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/Playoff_Teams/{team}", encoding="ISO-8859-1") as f:
            page = f.read()
        
        soup = BeautifulSoup(page, "html.parser")

        comment = next(c for c in soup.find_all(string=lambda text: isinstance(text, Comment)) if "per_poss_post" in c)
        table = BeautifulSoup(comment, "html.parser").find(id="per_poss_post")
        df = pd.read_html(StringIO(str(table)))[0]
        df["team_id"] = team
        playoff_per100_dfs.append(df)

playoff_per100_combined = pd.concat(playoff_per100_dfs, ignore_index=True)

playoff_per100_combined

#### Advanced

In [ ]:
all_advanced_dfs = []

for team in os.listdir(all_teams):
    if team.endswith(".html"):
        with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/All_Teams/{team}", encoding="ISO-8859-1") as f:
            page = f.read()
        
        soup = BeautifulSoup(page, "html.parser")

        comment = next(c for c in soup.find_all(string=lambda text: isinstance(text, Comment)) if 'id="advanced"' in c)
        table = BeautifulSoup(comment, "html.parser").find(id="advanced")
        df = pd.read_html(StringIO(str(table)))[0]
        df["team_id"] = team
        all_advanced_dfs.append(df)

all_advanced_combined = pd.concat(all_advanced_dfs, ignore_index=True)

all_advanced_combined

In [ ]:
playoff_advanced_dfs = []

for team in os.listdir(playoff_teams):
    if team.endswith(".html"):
        with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/Playoff_Teams/{team}", encoding="ISO-8859-1") as f:
            page = f.read()
        
        soup = BeautifulSoup(page, "html.parser")

        comment = next(c for c in soup.find_all(string=lambda text: isinstance(text, Comment)) if "advanced_post" in c)
        table = BeautifulSoup(comment, "html.parser").find(id="advanced_post")
        df = pd.read_html(StringIO(str(table)))[0]
        df["team_id"] = team
        playoff_advanced_dfs.append(df)

playoff_advanced_combined = pd.concat(playoff_advanced_dfs, ignore_index=True)

playoff_advanced_combined

#### Adjusted Shooting

In [ ]:
all_adjshooting_dfs = []

for team in os.listdir(all_teams):
    if team.endswith(".html"):
        with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/All_Teams/{team}", encoding="ISO-8859-1") as f:
            page = f.read()
        
        soup = BeautifulSoup(page, "html.parser")

        comment = next(c for c in soup.find_all(string=lambda text: isinstance(text, Comment)) if "adj_shooting" in c)
        table = BeautifulSoup(comment, "html.parser").find(id="adj_shooting")
        df = pd.read_html(StringIO(str(table)), header=1)[0]
        df["team_id"] = team
        all_adjshooting_dfs.append(df)

all_adjshooting_combined = pd.concat(all_adjshooting_dfs, ignore_index=True)

all_adjshooting_combined

In [ ]:
playoff_adjshooting_dfs = []

for team in os.listdir(playoff_teams):
    if team.endswith(".html"):
        with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/Playoff_Teams/{team}", encoding="ISO-8859-1") as f:
            page = f.read()
        
        soup = BeautifulSoup(page, "html.parser")

        comment = next(c for c in soup.find_all(string=lambda text: isinstance(text, Comment)) if "adj_shooting_post" in c)
        table = BeautifulSoup(comment, "html.parser").find(id="adj_shooting_post")
        df = pd.read_html(StringIO(str(table)), header=1)[0]
        df["team_id"] = team
        playoff_adjshooting_dfs.append(df)

playoff_adjshooting_combined = pd.concat(playoff_adjshooting_dfs, ignore_index=True)

playoff_adjshooting_combined

#### Shooting

In [ ]:
all_shooting_dfs = []

for team in os.listdir(all_teams):
    if team.endswith(".html"):
        with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/All_Teams/{team}", encoding="ISO-8859-1") as f:
            page = f.read()
        
        soup = BeautifulSoup(page, "html.parser")

        comment = next(c for c in soup.find_all(string=lambda text: isinstance(text, Comment)) if 'id="shooting"' in c)
        table = BeautifulSoup(comment, "html.parser").find(id="shooting")
        df = pd.read_html(StringIO(str(table)), header=1)[0]
        df["team_id"] = team
        all_shooting_dfs.append(df)

all_shooting_combined = pd.concat(all_shooting_dfs, ignore_index=True)

all_shooting_combined

In [ ]:
playoff_shooting_dfs = []

for team in os.listdir(playoff_teams):
    if team.endswith(".html"):
        with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/Playoff_Teams/{team}", encoding="ISO-8859-1") as f:
            page = f.read()
        
        soup = BeautifulSoup(page, "html.parser")

        comment = next(c for c in soup.find_all(string=lambda text: isinstance(text, Comment)) if 'id="shooting_post"' in c)
        table = BeautifulSoup(comment, "html.parser").find(id="shooting_post")
        df = pd.read_html(StringIO(str(table)), header=1)[0]
        df["team_id"] = team
        playoff_shooting_dfs.append(df)

playoff_shooting_combined = pd.concat(playoff_shooting_dfs, ignore_index=True)

playoff_shooting_combined

#### Play by Play

In [ ]:
all_pbp_dfs = []

for team in os.listdir(all_teams):
    if team.endswith(".html"):
        with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/All_Teams/{team}", encoding="ISO-8859-1") as f:
            page = f.read()
        
        soup = BeautifulSoup(page, "html.parser")

        comment = next(c for c in soup.find_all(string=lambda text: isinstance(text, Comment)) if "pbp_stats" in c)
        table = BeautifulSoup(comment, "html.parser").find(id="pbp_stats")
        df = pd.read_html(StringIO(str(table)), header=1)[0]
        df["team_id"] = team
        all_pbp_dfs.append(df)

all_pbp_combined = pd.concat(all_pbp_dfs, ignore_index=True)

all_pbp_combined

In [ ]:
playoff_pbp_dfs = []

for team in os.listdir(playoff_teams):
    if team.endswith(".html"):
        with open(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/Teams/Playoff_Teams/{team}", encoding="ISO-8859-1") as f:
            page = f.read()
        
        soup = BeautifulSoup(page, "html.parser")

        comment = next(c for c in soup.find_all(string=lambda text: isinstance(text, Comment)) if "pbp_stats_post" in c)
        table = BeautifulSoup(comment, "html.parser").find(id="pbp_stats_post")
        df = pd.read_html(StringIO(str(table)), header=1)[0]
        df["team_id"] = team
        playoff_pbp_dfs.append(df)

playoff_pbp_combined = pd.concat(playoff_pbp_dfs, ignore_index=True)

playoff_pbp_combined

### All Teams Export

In [ ]:
all_team_dfs_dict = {
    "pgs": all_pgs_combined,
    "misc": all_misc_combined,
    "totals": all_totals_combined,
    "per36": all_per36_combined,
    "per100": all_per100_combined,
    "advanced": all_advanced_combined,
    "adjshooting": all_adjshooting_combined,
    "shooting": all_shooting_combined,
    "pbp": all_pbp_combined
}

for name, df in all_team_dfs_dict.items():
    df.to_csv(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/{name}.csv", index=False)

In [ ]:
playoff_team_dfs_dict = {
    "playoff_pgs": playoff_pgs_combined,
    "playoff_totals": playoff_totals_combined,
    "playoff_per36": playoff_per36_combined,
    "playoff_per100": playoff_per100_combined,
    "playoff_advanced": playoff_advanced_combined,
    "playoff_adjshooting": playoff_adjshooting_combined,
    "playoff_shooting": playoff_shooting_combined,
    "playoff_pbp": playoff_pbp_combined
}

for name, df in playoff_team_dfs_dict.items():
    df.to_csv(f"/Users/camsmithers/Desktop/Camalytics/CamalyticsEnv/Projects/Sports/NBA/Data/{name}.csv", index=False)